# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [28]:
!pip freeze | grep gensim


gensim==4.2.0


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [29]:
import numpy as np


In [30]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [31]:
from gensim.models import Word2Vec

embedding_dim = 100

word2vec = Word2Vec(sentences=X_train, vector_size=embedding_dim, window=5, min_count=1, workers=4)


Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [32]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)


☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [33]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)


# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [34]:
import numpy as np

most_frequent_label = np.argmax(np.bincount(y_train))

baseline_accuracy = np.mean(y_test == most_frequent_label)

print("baseline accuracy", baseline_accuracy)


baseline accuracy 0.492


# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

model = Sequential()

model.add(Masking(mask_value=0.0, input_shape=(None, embedding_dim)))

model.add(LSTM(20, activation='tanh'))

model.add(Dense(10, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, None, 100)         0         
                                                                 
 lstm_2 (LSTM)               (None, 20)                9680      
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,901
Trainable params: 9,901
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    epochs=20,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping]
)


Epoch 1/20
63/63 [==============================] - 5s 45ms/step - loss: 0.6951 - accuracy: 0.4955 - val_loss: 0.6904 - val_accuracy: 0.5700
Epoch 2/20
63/63 [==============================] - 2s 36ms/step - loss: 0.6910 - accuracy: 0.5315 - val_loss: 0.6877 - val_accuracy: 0.5340
Epoch 3/20
63/63 [==============================] - 2s 36ms/step - loss: 0.6884 - accuracy: 0.5365 - val_loss: 0.6800 - val_accuracy: 0.5880
Epoch 4/20
63/63 [==============================] - 2s 36ms/step - loss: 0.6824 - accuracy: 0.5670 - val_loss: 0.6747 - val_accuracy: 0.5940
Epoch 5/20
63/63 [==============================] - 2s 36ms/step - loss: 0.6721 - accuracy: 0.5805 - val_loss: 0.6555 - val_accuracy: 0.6200
Epoch 6/20
63/63 [==============================] - 2s 37ms/step - loss: 0.6665 - accuracy: 0.6010 - val_loss: 0.6519 - val_accuracy: 0.6380
Epoch 7/20
63/63 [==============================] - 2s 35ms/step - loss: 0.6595 - accuracy: 0.6125 - val_loss: 0.6489 - val_accuracy: 0.6500
Epoch 8/20
63

❓ **Question** ❓ Evaluate your model on the test set

In [37]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print("test accuracy", test_accuracy)


79/79 [==============================] - 1s 9ms/step - loss: 0.6443 - accuracy: 0.6412
test accuracy 0.6412000060081482


# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [38]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [52]:
word2vec_transfer = api.load("glove-wiki-gigaword-100")


[==================================================] 100.0% 128.1/128.1MB downloaded


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [54]:
print("vocab size", len(word2vec_transfer.index_to_key))
print("embedding space size", word2vec_transfer.vector_size)


vocab size 400000
embedding space size 100


❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [57]:
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed


❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [58]:
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)


❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

model_transfer = Sequential()

model_transfer.add(Masking(mask_value=0.0, input_shape=(None, word2vec_transfer.vector_size)))

model_transfer.add(LSTM(20, activation='tanh'))

model_transfer.add(Dense(10, activation='relu'))

model_transfer.add(Dense(1, activation='sigmoid'))

model_transfer.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model_transfer.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_7 (Masking)         (None, None, 100)         0         
                                                                 
 lstm_7 (LSTM)               (None, 20)                9680      
                                                                 
 dense_14 (Dense)            (None, 10)                210       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,901
Trainable params: 9,901
Non-trainable params: 0
_________________________________________________________________


In [60]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_transfer = model_transfer.fit(
    X_train_pad_2, y_train,
    epochs=10,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping]
)


Epoch 1/10
63/63 [==============================] - 5s 46ms/step - loss: 0.6895 - accuracy: 0.5385 - val_loss: 0.6822 - val_accuracy: 0.6160
Epoch 2/10
63/63 [==============================] - 2s 35ms/step - loss: 0.6728 - accuracy: 0.6110 - val_loss: 0.6473 - val_accuracy: 0.6460
Epoch 3/10
63/63 [==============================] - 2s 35ms/step - loss: 0.6430 - accuracy: 0.6360 - val_loss: 0.6159 - val_accuracy: 0.6900
Epoch 4/10
63/63 [==============================] - 2s 35ms/step - loss: 0.6112 - accuracy: 0.6740 - val_loss: 0.5890 - val_accuracy: 0.6940
Epoch 5/10
63/63 [==============================] - 2s 35ms/step - loss: 0.5918 - accuracy: 0.6990 - val_loss: 0.5670 - val_accuracy: 0.7180
Epoch 6/10
63/63 [==============================] - 2s 35ms/step - loss: 0.5699 - accuracy: 0.7125 - val_loss: 0.5459 - val_accuracy: 0.7320
Epoch 7/10
63/63 [==============================] - 2s 35ms/step - loss: 0.5566 - accuracy: 0.7200 - val_loss: 0.5438 - val_accuracy: 0.7460
Epoch 8/10
63

In [49]:
test_loss_transfer, test_accuracy_transfer = model_transfer.evaluate(X_test_pad_2, y_test)
print("test accuracy with transfer learning", test_accuracy_transfer)


79/79 [==============================] - 1s 7ms/step - loss: 0.5880 - accuracy: 0.6972
test accuracy with transfer learning 0.6972000002861023


In [61]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')


The accuracy evaluated on the test set is of 49.080%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously